## Reference
- [Recurrent Neural Networks in Tensorflow II](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html)

## Imports, Task and data

In [1]:
import numpy as np
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import time
import os
import urllib.request

import seaborn; seaborn.set()

In [2]:
# Load data
file_url = 'https://raw.githubusercontent.com/samim23/obama-rnn/master/input.txt'
file_name = 'obama_speech.txt'
if not os.path.exists(file_name):
    urllib.request.urlretrieve(file_url, file_name)
    
with open(file_name, 'r') as f:
    raw_data = f.read()
    print(raw_data[:30] + ".....")
    print("Data length: ", len(raw_data))

To Chip, Kathy, and Nancy, who.....
Data length:  4224143


In [3]:
# process data - vocab for char and transformed vocab to idx.

vocab = set(raw_data)
vocab_size = len(vocab)
idx_to_vocab = dict(enumerate(vocab))
vocab_to_idx = dict(zip(idx_to_vocab.values(), idx_to_vocab.keys()))

data = [vocab_to_idx[c] for c in raw_data]
del raw_data
print(data[:10])

[46, 40, 107, 17, 57, 62, 93, 20, 107, 64]


In [4]:
# utility functions
def gen_epochs(n, num_steps, batch_size):
    for i in range(n):
        yield ptb_iterator(data, batch_size, num_steps)
    
def ptb_iterator(raw_data, batch_size, num_steps):
    raw_data = np.array(raw_data, dtype=np.int32)
    
    data_len = len(raw_data)
    batch_len = data_len // batch_size
    data = np.zeros([batch_size, batch_len], dtype=np.int32)
    for i in range(batch_size):
        data[i] = raw_data[batch_len*i:batch_len*(i+1)]
        
    epoch_size = (batch_len - 1) // num_steps
    
    if epoch_size == 0:
        raise ValueError("epoch size == 0, decrease batch_size or num_steps")
        
    for i in range(epoch_size):
        x = data[:, i*num_steps: (i+1)*num_steps]
        y = data[:, i*num_steps+1: (i+1)*num_steps+1]
        yield (x, y)
        
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
def train(g, num_epochs, num_steps=200, batch_size=32, verbose=True, 
          verbose_step=1000, save=False):
    tf.set_random_seed(2345)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        traininig_losses = []
        for idx, epoch in enumerate(gen_epochs(num_epochs, num_steps, batch_size)):
            traininig_loss = 0
            steps = 0
            traininig_state = None
            for X, y in epoch:
                steps += 1
                feed_dict ={g['x']: X, g['y']: y}
                if traininig_state is not None:
                    feed_dict[g['init_state']] = traininig_state
                traininig_loss_, traininig_state, _ = sess.run(
                    [g['total_loss'], g['final_state'], g['train_step']], feed_dict)
                traininig_loss += traininig_loss_
                
            if verbose:
                print("Average trainig loss for Epoch", idx, ":", traininig_loss/steps)
            traininig_losses.append(traininig_loss/steps)
            
        if isinstance(save, str):
            g['saver'].save(sess, save)
    
    return traininig_losses

## Model

In [5]:
def build_graph(state_size=100, num_classes=vocab_size, batch_size=32, num_steps=200,
                num_layers=3, learning_rate=1e-4):
    
    reset_graph()
    
    X = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
    y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placehoder')
    
    embeddings = tf.get_variable('embedding_matrix', [num_classes, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, X)
    
    cell = tf.nn.rnn_cell.GRUCell(state_size)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)
    
    init_state = cell.zero_state(batch_size, tf.float32)
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)
    
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
        
    # reshape rnn_outputs and y
    rnn_outputs = tf.reshape(rnn_outputs, [-1, state_size])
    y_reshaped = tf.reshape(y, [-1])
    
    logits = tf.matmul(rnn_outputs, W) + b
    predictions = tf.nn.softmax(logits)
    total_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_reshaped))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
    
    return {
        "x": X,
        "y": y,
        "init_state": init_state,
        "final_state": final_state,
        "total_loss": total_loss,
        "train_step": train_step,
        "prediction": predictions,
        "saver": tf.train.Saver()
    }

## Training

In [6]:
train_epochs = 20
train_num_steps = 80

g = build_graph(num_steps=train_num_steps)
t = time.time()

checkpoint_dir = "GRU_" + str(train_epochs) + "_epochs"
losses = train(g, train_epochs, num_steps=train_num_steps, save=checkpoint_dir)
print("It took", time.time() - t, "seconds to train for 20 epochs.")
print("The average loss on the final epoch was:", losses[-1])

Average trainig loss for Epoch 0 : 2.79619301406
Average trainig loss for Epoch 1 : 2.14077008927
Average trainig loss for Epoch 2 : 1.93491003593
Average trainig loss for Epoch 3 : 1.8059073696
Average trainig loss for Epoch 4 : 1.71586091396
Average trainig loss for Epoch 5 : 1.64723603921
Average trainig loss for Epoch 6 : 1.59187777057
Average trainig loss for Epoch 7 : 1.54619720502
Average trainig loss for Epoch 8 : 1.50815572074
Average trainig loss for Epoch 9 : 1.47590758338
Average trainig loss for Epoch 10 : 1.44818767475
Average trainig loss for Epoch 11 : 1.42415895744
Average trainig loss for Epoch 12 : 1.40319405483
Average trainig loss for Epoch 13 : 1.38476076625
Average trainig loss for Epoch 14 : 1.3683832847
Average trainig loss for Epoch 15 : 1.35365466118
Average trainig loss for Epoch 16 : 1.34019100811
Average trainig loss for Epoch 17 : 1.32801722274
Average trainig loss for Epoch 18 : 1.31693589572
Average trainig loss for Epoch 19 : 1.30677704168
It took 6809

## Generating text

In [7]:
def generate_characters(g, checkpoint, num_chars, prompt='A', pick_top_chars=None):
    # Accepts a current character, inital state
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        g['saver'].restore(sess, checkpoint)
        
        state = None
        current_char = vocab_to_idx[prompt]
        chars = [current_char]
        
        for i in range(num_chars):
            if state is not None:
                feed_dict = {g['x']: [[current_char]], g['init_state']: state}
            else:
                feed_dict = {g['x']: [[current_char]]}
                
        preds, state = sess.run([g['prediction'], g['final_state']], feed_dict)
        
        if pick_top_chars is not None:
            p = np.squeeze(preds)
            p[np.argsort(p)[:-pick_top_chars]] = 0
            p = 0 / np.sum(p)
            current_char = np.random.choice(vocab_size, 1, p=p)[0]
        else:
            current_char = np.random.choice(vocab_size, 1, p=np.squeeze(preds))[0]
            
        chars.append(current_char)
        
    chars = map(lambda x: idx_to_vocab[x], chars)
    print("".join(chars))
    return ("".join(chars))

In [8]:
g = build_graph(num_steps=1, batch_size=1)
generate_characters(g, checkpoint_dir, 750, prompt='A', pick_top_chars=5)

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for GRU_20_epochs
	 [[Node: save/RestoreV2_3 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_3/tensor_names, save/RestoreV2_3/shape_and_slices)]]

Caused by op 'save/RestoreV2_3', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Ver